# 0.0 IMPORTS AND DATA LOADING

In [179]:
import findspark
findspark.init()
from pyspark.sql import SparkSession 
from pyspark.sql.functions import isnan, when, count, col, sum, expr
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.stat import Correlation
from pyspark.ml.classification import LogisticRegression
from imblearn.over_sampling import RandomOverSampler

In [86]:
spark = SparkSession.builder.getOrCreate()

df = spark.read.csv('Fraud.csv',inferSchema=True,header=True)

df = df.drop(df.isFlaggedFraud)

# 1.0 DATA DESCRIPTION

In [87]:
df.printSchema()

root
 |-- step: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: integer (nullable = true)



In [88]:
df.show(5)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|
|   1|CASH_OUT|   181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|
|   1| PAYMENT|11668.14|C2048537720|      41554.0|      29885.86|M1230701703|           0.0|           0.0|      0|
+----+--------+--------+-----------+-------------+--------------+-------

In [89]:
print(df.count())
print(len(df.columns))

6362620
10


In [90]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+
|step|type|amount|nameOrig|oldbalanceOrg|newbalanceOrig|nameDest|oldbalanceDest|newbalanceDest|isFraud|
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+
|   0|   0|     0|       0|            0|             0|       0|             0|             0|      0|
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+



In [91]:
df.summary().toPandas()

,summary,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,count,6362620,6362620,6362620,6362620,6362620,6362620,6362620,6362620,6362620,6362620
1,mean,243.39724563151657,None,179861.90354913048,None,833883.1040744851,855113.6685785787,None,1100701.6665196505,1224996.3982019336,0.001290820448180152
2,stddev,142.3319710491305,None,603858.2314629437,None,2888242.6730375397,2924048.502954241,None,3399180.1129944543,3674128.9421196654,0.03590479680160438
3,min,1,CASH_IN,0.0,C1000000639,0.0,0.0,C1000004082,0.0,0.0,0
4,25%,156,None,13390.35,None,0.0,0.0,None,0.0,0.0,0
5,50%,239,None,74864.38,None,14211.23,0.0,None,132623.66,214605.81,0
6,75%,335,None,208722.05,None,107294.0,144232.62,None,942942.08,1111684.56,0
7,max,743,TRANSFER,9.244551664E7,C999999784,5.958504037E7,4.958504037E7,M999999784,3.5601588935E8,3.5617927892E8,1


# 2.0 FEATURE ENGINEERING

In [92]:
df = df.withColumn('changeOrig', (df.newbalanceOrig - df.oldbalanceOrg))

df = df.withColumn('changeDest', (df.newbalanceDest - df.oldbalanceDest))

In [93]:
df.limit(5).toPandas()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,changeOrig,changeDest
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,-9839.64,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,-1864.28,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,-181.00,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,-181.00,-21182.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,-11668.14,0.0


# 3.0 FILTERING DATA

In [94]:
df = df.drop(*['step','nameOrig','nameDest'])

In [95]:
df.limit(5).toPandas()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,changeOrig,changeDest
0,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0,-9839.64,0.0
1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0,-1864.28,0.0
2,TRANSFER,181.00,181.0,0.00,0.0,0.0,1,-181.00,0.0
3,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1,-181.00,-21182.0
4,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0,-11668.14,0.0


# 4.0 EXPLORATORY DATA ANALYSIS

In [96]:
df.groupBy('isFraud').count().show()

+-------+-------+
|isFraud|  count|
+-------+-------+
|      1|   8213|
|      0|6354407|
+-------+-------+



In [97]:
df.groupBy('type').sum('isFraud').show()

+--------+------------+
|    type|sum(isFraud)|
+--------+------------+
|TRANSFER|        4097|
| CASH_IN|           0|
|CASH_OUT|        4116|
| PAYMENT|           0|
|   DEBIT|           0|
+--------+------------+



In [98]:
df.groupBy('type').count().show()

+--------+-------+
|    type|  count|
+--------+-------+
|TRANSFER| 532909|
| CASH_IN|1399284|
|CASH_OUT|2237500|
| PAYMENT|2151495|
|   DEBIT|  41432|
+--------+-------+



# 5.0 DATA PREPARATION

In [202]:
typeIndexer = StringIndexer(inputCol='type', outputCol='typeIndex')

df5 = typeIndexer.fit(df).transform(df)

one_hot_encoder = OneHotEncoder(inputCol='typeIndex', outputCol='typeOneHot')

# Use the encoder to transform the indexed dataframe
one_hot_fitted = one_hot_encoder.fit(df5)
df5 = one_hot_fitted.transform(df5)

In [203]:
df5.limit(5).toPandas()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,changeOrig,changeDest,typeIndex,typeOneHot
0,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0,-9839.64,0.0,1.0,"(0.0, 1.0, 0.0, 0.0)"
1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0,-1864.28,0.0,1.0,"(0.0, 1.0, 0.0, 0.0)"
2,TRANSFER,181.00,181.0,0.00,0.0,0.0,1,-181.00,0.0,3.0,"(0.0, 0.0, 0.0, 1.0)"
3,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1,-181.00,-21182.0,0.0,"(1.0, 0.0, 0.0, 0.0)"
4,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0,-11668.14,0.0,1.0,"(0.0, 1.0, 0.0, 0.0)"


In [204]:
df5 = df5.drop('type','typeIndex')

In [205]:
train, test = df5.randomSplit([0.7,0.3],seed=42)

ones = train.filter(col("isFraud") == 1)
zeros = train.filter(col("isFraud") == 0)
print(ones.count())
print(zeros.count())
n = 3
for i in range(n-1):
    ones = ones.union(ones)
print(ones.count())
print(zeros.count())

5732
4448458
22928
4448458


In [206]:
zeros = zeros.limit(ones.count())
train = zeros.union(ones)
print(ones.count())
print(zeros.count())
print(train.count())

22928
22928
45856


In [207]:
train.columns

['amount',
 'oldbalanceOrg',
 'newbalanceOrig',
 'oldbalanceDest',
 'newbalanceDest',
 'isFraud',
 'changeOrig',
 'changeDest',
 'typeOneHot']

In [208]:
assembler = VectorAssembler(inputCols=['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest',
                                       'changeOrig','changeDest'],
                           outputCol='features')

In [209]:
train = assembler.transform(train)
train = train.select(col('features'),col('isFraud'))

In [210]:
test = assembler.transform(test)
test = test.select(col('features'),col('isFraud'))

# 6.0 MACHINE LEARNING MODELLING

In [211]:
lr = LogisticRegression(labelCol='isFraud')
lr.setThreshold(0.2)
lrn = lr.fit(train)
lr_pred = lrn.transform(test)

lr_pred = lr_pred.withColumn("label",col("isFraud").cast('double'))

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
evaluator = BinaryClassificationEvaluator(labelCol='isFraud')
predrdd = lr_pred['label','prediction'].rdd
metrics = MulticlassMetrics(predrdd)
print('Test Area Under ROC', evaluator.evaluate(lr_pred))
print('Precision',metrics.precision(label=1))
print('Recall',metrics.recall(label=1))
print('F1-Score',metrics.fMeasure(1.0))

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'isFraud')
rf.setThresholds([0.3, 0.2])
rfModel = rf.fit(train)
rf_pred = rfModel.transform(test)

rf_pred = rf_pred.withColumn("label",col("isFraud").cast('double'))

In [ ]:
predrdd = rf_pred['label','prediction'].rdd
metrics = MulticlassMetrics(predrdd)
print('Test Area Under ROC', evaluator.evaluate(rf_pred))
print('Precision',metrics.precision(label=1))
print('Recall',metrics.recall(label=1))
print('F1-Score',metrics.fMeasure(1.0))